## Tabela Classificação do Sport

In [ ]:
import json
import io
import base64
import html
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

from highlight_text import fig_text
from mplsoccer import Bumpy, FontManager, add_image

In [ ]:
df_total_jogos = pd.DataFrame(total_jogos)

In [ ]:
def calcular_classificacao_sport_por_rodada(df):
    def calcular_estatisticas(df):
        estatisticas = {}
        
        for _, row in df.iterrows():
            mandante = row['Mandante']
            visitante = row['Visitante']
            placar_mandante = row['Placar Mandante']
            placar_visitante = row['Placar Visitante']
            
            if mandante not in estatisticas:
                estatisticas[mandante] = {'P': 0, 'V': 0, 'E': 0, 'D': 0, 'Gols': '0:0', 'J': 0}
            if visitante not in estatisticas:
                estatisticas[visitante] = {'P': 0, 'V': 0, 'E': 0, 'D': 0, 'Gols': '0:0', 'J': 0}
            
            mandante_gols = estatisticas[mandante]['Gols']
            visitante_gols = estatisticas[visitante]['Gols']
            mandante_realizados, mandante_sofridos = map(int, mandante_gols.split(':'))
            visitante_realizados, visitante_sofridos = map(int, visitante_gols.split(':'))
            
            mandante_realizados += placar_mandante
            mandante_sofridos += placar_visitante
            visitante_realizados += placar_visitante
            visitante_sofridos += placar_mandante
            
            estatisticas[mandante]['Gols'] = f"{mandante_realizados}:{mandante_sofridos}"
            estatisticas[visitante]['Gols'] = f"{visitante_realizados}:{visitante_sofridos}"
            
            if placar_mandante > placar_visitante:
                estatisticas[mandante]['P'] += 3
                estatisticas[mandante]['V'] += 1
                estatisticas[visitante]['D'] += 1
            elif placar_mandante < placar_visitante:
                estatisticas[visitante]['P'] += 3
                estatisticas[visitante]['V'] += 1
                estatisticas[mandante]['D'] += 1
            else:
                estatisticas[mandante]['P'] += 1
                estatisticas[visitante]['P'] += 1
                estatisticas[mandante]['E'] += 1
                estatisticas[visitante]['E'] += 1
            
            estatisticas[mandante]['J'] += 1
            estatisticas[visitante]['J'] += 1
        
        return estatisticas

    rodadas = []
    classificacoes = []

    for rodada in sorted(df['Rodada'].unique()):
        df_atual = df[df['Rodada'] <= rodada]
        
        estatisticas_times = calcular_estatisticas(df_atual)
        
        df_estatisticas = pd.DataFrame.from_dict(estatisticas_times, orient='index').reset_index()
        df_estatisticas.rename(columns={'index': 'Time'}, inplace=True)
        df_estatisticas = df_estatisticas[['Time', 'P', 'J', 'V', 'E', 'D', 'Gols']]
        df_estatisticas.sort_values(by='P', ascending=False, inplace=True)
        df_estatisticas['#'] = range(1, len(df_estatisticas) + 1)
        df_estatisticas = df_estatisticas[['#', 'Time', 'P', 'J', 'V', 'E', 'D', 'Gols']]
        
        sport_classificacao = df_estatisticas[df_estatisticas['Time'] == 'Sport']
        if not sport_classificacao.empty:
            rodadas.append(rodada)
            classificacoes.append(sport_classificacao['#'].values[0])
        else:
            rodadas.append(rodada)
            classificacoes.append(None)  # Caso o Sport não tenha jogado na rodada
    
    df_classificacao_sport = pd.DataFrame({'Rodada': rodadas, 'Classificação': classificacoes})
    
    return df_classificacao_sport

df_classificacao_sport = calcular_classificacao_sport_por_rodada(df_total_jogos)

In [ ]:
font_normal = FontManager('https://raw.githubusercontent.com/googlefonts/roboto/'
                          'main/src/hinted/Roboto-Regular.ttf')
font_bold = FontManager('https://raw.githubusercontent.com/google/fonts/main/apache/robotoslab/'
                        'RobotoSlab[wght].ttf')

In [ ]:
# Dados de exemplo (substitua pelos seus dados reais)
match_day = df_classificacao_sport [ 'Rodada']
positions = df_classificacao_sport [ 'Classificação']

In [ ]:
# Season dict format para bumpy plot
season_dict = {
    "Sport Recife": positions 
}

# Configurar o dicionário de destaque
highlight_dict = {
    "Sport Recife": "crimson"
}

In [ ]:
bumpy = Bumpy(
    scatter_color="#282A2C",  # Cor dos marcadores
    line_color="#252525",  # Cor das linhas
    rotate_xticks=90,  # Rotacionar xticks em 90 graus
    ticklabel_size=17,  # Tamanho das labels
    label_size=30,  # Tamanho das labels principais
    scatter_primary='D',  # Marcador a ser usado
    show_right=True,  # Mostrar posição no lado direito
    plot_labels=True,  # Plotar as labels
    alignment_yvalue=0.1,  # Alinhamento das labels no eixo y
    alignment_xvalue=0.065  # Alinhamento das labels no eixo x
)

fig, ax = bumpy.plot(
    x_list=match_day,  # Dia da rodada
    y_list=np.linspace(1, 10, 10).astype(int),  # Valores de posição de 1 a 10
    values=season_dict,  # Valores contendo posições para cada time
    secondary_alpha=0.5,  # Valor alpha para linhas/marcadores não sombreado
    highlight_dict=highlight_dict,  # Time para ser destacado com suas cores
    figsize=(20, 12),  # Ajuste o tamanho da figura para um formato mais compacto
    x_label='Rodada', y_label='Posição',  # Nome dos labels
    ylim=(0, 11),  # Limite do eixo y ajustado para incluir o valor 10
    lw=2.5,  # Largura da linha de conexão
    fontproperties=FontProperties(size=17)  # Propriedades da fonte para labels
)


# Ajustar layout
plt.tight_layout(pad=2.0)

buf = io.BytesIO()
fig.savefig(buf, format='png', dpi=150, bbox_inches='tight') #salvo neste formato para inserir imagem no HTML posteriormente
buf.seek(0)
img_str = base64.b64encode(buf.read()).decode('utf-8')

# Fechar o gráfico
plt.close(fig)